In [1]:
%matplotlib notebook

# generic packages
import sys
import re, numpy as np, pandas as pd
from pprint import pprint
from matplotlib.ticker import PercentFormatter

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 1)

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# graphing, vis stuff
import matplotlib.pyplot as plt

# additional libraries
from collections import Counter



# 1 Region and Domain

# 2 Research Question

# 3 Links

In [6]:
# competitive contracts - City of Toronto

CC_URL = "https://wx.toronto.ca/inter/pmmd/callawards.nsf/postedawards?OpenView"
CC_URL2 = "https://wx.toronto.ca/inter/pmmd/callawards.nsf/postedawards?OpenView&Start=3.8.1"

# non-competitive contracts - City of Toronto

NC_URL2 = "https://wx.toronto.ca/inter/pmmd/solesource.nsf/posted?OpenView&Start=1000&ExpandView"
NC_URL3 = "https://wx.toronto.ca/inter/pmmd/solesource.nsf/posted?OpenView&Start=1999&ExpandView"

In [8]:
# WORK THROUGH ... https://realpython.com/beautiful-soup-web-scraper-python/#step-2-scrape-html-content-from-a-page

from bs4 import BeautifulSoup
import requests

NC_URL = "https://wx.toronto.ca/inter/pmmd/solesource.nsf/posted?OpenView&Start=1&ExpandView"
page = requests.get(NC_URL)

soup = BeautifulSoup(page.content, "html.parser")



# 4 Image

# 5 Discussion